In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/Cat_vs_Dog/data_1500.zip'

Archive:  /content/drive/MyDrive/Cat_vs_Dog/data_1500.zip
   creating: data_1500/
   creating: data_1500/training_set/
   creating: data_1500/training_set/cats/
  inflating: data_1500/training_set/cats/cat.0.jpg  
  inflating: data_1500/training_set/cats/cat.1.jpg  
  inflating: data_1500/training_set/cats/cat.10.jpg  
  inflating: data_1500/training_set/cats/cat.100.jpg  
  inflating: data_1500/training_set/cats/cat.1000.jpg  
  inflating: data_1500/training_set/cats/cat.1001.jpg  
  inflating: data_1500/training_set/cats/cat.1002.jpg  
  inflating: data_1500/training_set/cats/cat.1003.jpg  
  inflating: data_1500/training_set/cats/cat.1004.jpg  
  inflating: data_1500/training_set/cats/cat.1005.jpg  
  inflating: data_1500/training_set/cats/cat.1006.jpg  
  inflating: data_1500/training_set/cats/cat.1007.jpg  
  inflating: data_1500/training_set/cats/cat.1008.jpg  
  inflating: data_1500/training_set/cats/cat.1009.jpg  
  inflating: data_1500/training_set/cats/cat.101.jpg  
  inflati

In [3]:
# 导入包
import cv2 as cv
import numpy as np
import glob
import os
from tqdm import tqdm

In [4]:
# 定义一共函数，整合数据，需要传入图像的路径path，和图像的标签label
# 每执行一次函数，会执行整合一种类型的标签集。
def loadData(oriPath, label):
#     继续编辑原代码块，猫狗图像按照数据集类型独立存放，因此可以使用 glob 函数遍历同种类图像。
#     函数会返回该路径下所有符合条件的文件路径。
    allImagePath = glob.glob(os.path.join(oriPath, '*.jpg'))
#     建立一个空数组，把数据压缩成224*224
    x = np.empty([len(allImagePath), 224,224,3])
#     二分类问题，用一维数组，存放标签
    y = np.empty([0])
#     遍历当前路径下的图像，在每一次循环中使用 cv.imread 读取图像。
    for idx in tqdm(range(len(allImagePath))):
        imagePath = allImagePath[idx]
        image = cv.imread(imagePath)
#openCV 直接读取的图像是 BGR 三通道的，因此使用 cv.cvtColor 函数将 BGR 三通道的图像转换为 RGB 三通道的图像。
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
#原始图像的分辨率大小不一致，无法直接存入数据集合中。因此使用 cv.resize 函数对图像进行放缩。
        image = cv.resize(image, (224, 224))
# 将image存进x数组。
        x[idx,:] = image
# 使用 np.linspace 函数定义 numpy 数组，并用 label 标签进行填充。
        y = np.linspace(label,label,x.shape[0])
    return x, y

    

In [5]:
# 调用 loadData 函数分别对训练集和测试集中的猫狗图片进行整合。
# trainCatX, trainCatY = loadData('/content/data_smoll/training_set/cats', 0)
# trainDogX, trainDogY = loadData('/content/data_smoll/training_set/dogs', 1)
evalCatX, evalCatY = loadData('/content/data_1500/test_set/cats', 0)
evalDogX, evalDogY = loadData('/content/data_1500/test_set/dogs', 1)

100%|██████████| 1500/1500 [00:05<00:00, 263.77it/s]


In [6]:
# 使用 np.concatenate 函数分别将训练集的猫狗和测试集的猫狗数据集合并。axis=0为按行合并。
# trainData = np.concatenate((trainCatX, trainDogX), axis = 0)
# trainLabel = np.concatenate((trainCatY, trainDogY), axis = 0)
evalData = np.concatenate((evalCatX, evalDogX), axis = 0)
evalLabel = np.concatenate((evalCatY, evalDogY), axis = 0)

In [7]:
# 测试数据整合情况
# trainImage = trainData[233]
testImage = evalData[233]

In [8]:
# 在图片导出时 OpenCV 仍然以 BGR 的方式理解，因此需要再将其转换为 RGB。
# 并且整合后的数据集类型为 float64，
# 需要使用 trainImage.astype 和 testImage.astype 函数将其转换为 uint8 后再执行 cv.cvtColor 函数。
# trainImage = cv.cvtColor(trainImage.astype(np.uint8), cv.COLOR_RGB2BGR)
testImage = cv.cvtColor(testImage.astype(np.uint8), cv.COLOR_RGB2BGR)
# cv.imwrite('test1.jpg', trainImage)
cv.imwrite('test3.jpg', testImage)

True

In [9]:
# 把数据导出到data目录，方便后续操作
# np.save('/content/drive/MyDrive/Cat_vs_Dog/trainData.npy', trainData)
# np.save('/content/drive/MyDrive/Cat_vs_Dog/trainLabel.npy', trainLabel)
np.save('/content/drive/MyDrive/Cat_vs_Dog/evalData.npy', evalData)
np.save('/content/drive/MyDrive/Cat_vs_Dog/evalLabel.npy', evalLabel)

In [ ]:
# 模型选择使用MobielNet，
# 对于 MobileNet 模型来说，该模型要求传入的数据为范围在 -1~1 之间的浮点数。
# 前面的图像数据都是（0，255），而且是整型的
# 因此，在进行 Fine Tune 之前，我们需要对数据进行 Normalize 操作。